# Tugas 1
## Crawling Website

Rafly Faldiansyah Putra

210411100063

### Pengertian Crawling

Crawling data adalah proses pengambilan data secara otomatis dari website oleh mesin pencari. Ketika mesin pencari mengunjungi suatu website, ia akan mengikuti semua tautan yang ada di halaman tersebut dan mengambil informasi dari setiap halaman yang dikunjungi. Hasilnya, mesin pencari akan memiliki indeks data yang sangat besar yang dapat digunakan untuk menyajikan hasil informasi data yang dibutuhkan.


In [ ]:
!pip install requests beautifulsoup4 pandas

#install library untuk mengambil konten di dalam website

### 1. Melakukan import library yang dibutuhkan

In [2]:
# Importing Libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random


requests: Digunakan untuk melakukan HTTP request ke sebuah URL dan mendapatkan konten halaman web.

BeautifulSoup: Digunakan untuk mem-parsing HTML dan mengekstrak data dari halaman web.

pandas: Digunakan untuk mengolah data dan menyimpan hasil ekstraksi ke dalam bentuk DataFrame.

### 2. Mengambil Halaman Website dan Memeriksa status request

In [3]:
# Daftar user agents yang akan dirotasi
USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0',
]
# Fungsi untuk mengambil dan mem-parsing HTML dari URL
def get_soup(url):
    headers = {
        'User-Agent': random.choice(USER_AGENTS)
    }
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return BeautifulSoup(response.text, 'html.parser')
        else:
            print(f"Error {response.status_code} saat mengambil {url}")
            return None
    except Exception as e:
        print(f"Error: {e}")
        return None


### 3. Menemukan dan Mengekstrak 10 Berita

In [5]:
# Fungsi untuk mendapatkan detail artikel dari halaman detail
def get_article_details(detail_url, category_name):
    detail_soup = get_soup(detail_url)
    if detail_soup:
        # Ambil konten dari div dengan class 'read__content'
        content_div = detail_soup.find('div', class_='read__content')
        content = ' '.join([p.text for p in content_div.find_all('p')]) if content_div else 'Tidak ada isi berita'
        
        # Ambil tanggal dari div dengan class 'read__time'
        date_tag = detail_soup.find('div', class_='read__time')
        date = date_tag.text.strip().split('-')[-1].strip() if date_tag else 'Tidak ada tanggal'
        
        title_tag = detail_soup.find('h1')
        title = title_tag.text.strip() if title_tag else 'Tidak ada judul'
        
        return {
            'judul': title,
            'isi_berita': content,
            'tanggal': date,
            'kategori': category_name,
        }
    return None

# Fungsi untuk mendapatkan artikel dari suatu kategori
def get_articles(category_url, category_name, max_articles=10):
    articles = []
    page = 1
    while len(articles) < max_articles:
        url = f'{category_url}?page={page}'
        print(f"Mengambil: {url}")
        soup = get_soup(url)
        if soup is None:
            break
        article_list = soup.find_all('h3', class_='article__title')
        if not article_list:
            print(f"Tidak ada artikel ditemukan di halaman {page}.")
            break
        for article in article_list:
            if len(articles) >= max_articles:
                break
            title_tag = article.find('a')
            detail_url = title_tag['href'] if title_tag else None
            if detail_url:
                article_details = get_article_details(detail_url, category_name)
                if article_details:
                    articles.append(article_details)
        
        sleep_time = random.uniform(3, 6)
        time.sleep(sleep_time)
        
        page += 1
    return articles

# URL Kategori Kompas
categories = {
    'Olahraga': 'https://www.kompas.com/tag/olahraga',
    'Politik': 'https://www.kompas.com/tag/politik',
}

# Mengumpulkan semua data
all_articles = []
for category_name, category_url in categories.items():
    print(f"Menambang kategori {category_name}...")
    articles = get_articles(category_url, category_name, max_articles=10)
    all_articles.extend(articles)
# Simpan ke dalam DataFrame
df = pd.DataFrame(all_articles)

# Simpan ke dalam file CSV tanpa kolom 'url'
df.to_csv('kompas_articles.csv', index=False)


Menambang kategori Olahraga...
Mengambil: https://www.kompas.com/tag/olahraga?page=1
Menambang kategori Politik...
Mengambil: https://www.kompas.com/tag/politik?page=1


### 4. Menampilkan data yang di crowling ke dalam tabel

In [7]:
# Tampilkan 10 data acak dalam bentuk tabel
print(df.sample(n=10))

print("Proses penambangan data selesai, data tersimpan dalam 'kompas_articles.csv' dan 10 data pertama ditampilkan.")
from IPython.display import display

# Tampilkan 10 data acak dalam bentuk tabel
display(df.sample(n=10))


                                                judul  \
9   Menpora: Peparnas XVII Solo, Olahraga untuk Se...   
14                    Ekonomi Hijau dan Politik Kotor   
3   Potret Reijnders-Hilgers Berseragam Indonesia,...   
11  Ridwan Kamil Curhat Pengalaman Mengecewakan ya...   
13  Survei Litbang "Kompas": Sektor Politik dan Ke...   
1   Sama-sama Ajang Olahraga Tingkat Nasional, Apa...   
0   Keluar Rumah Hendak Olahraga, Anak Polisi di P...   
2   Wanita 54 Tahun Berhasil Turunkan Berat Badan ...   
10  Terima Kunjungan Elite PKS, Prabowo: Teman Lam...   
15  Jokowi dan Kendaraan Politiknya Jelang Pensiun...   

                                           isi_berita                tanggal  \
9   SOLO, KOMPAS.com - Menteri Pemuda dan Olahraga...  07/10/2024, 07:47 WIB   
14  TAHUN 2024 menjadi penanda penting dalam perja...  11/10/2024, 08:45 WIB   
3   Dua pemain baru Timnas Indonesia Eliano Reijnd...      Tidak ada tanggal   
11  JAKARTA, KOMPAS.com - Calon gubernur Jakarta n..

,judul,isi_berita,tanggal,kategori
5,Studi Ungkap Durasi Olahraga agar Tensi Stabil...,KOMPAS.com - Olahraga membantu meningkatkan ke...,"09/10/2024, 05:45 WIB",Olahraga
17,"Port Visit KRI WSH-991, Perkuat Politik Bebas ...",Pangkoarmada RI Laksamana Madya Denih Hendrata...,Tidak ada tanggal,Politik
14,Ekonomi Hijau dan Politik Kotor,TAHUN 2024 menjadi penanda penting dalam perja...,"11/10/2024, 08:45 WIB",Politik
1,"Sama-sama Ajang Olahraga Tingkat Nasional, Apa...",KOMPAS.com - Pekan Paralimpiade Nasional (Pepa...,"10/10/2024, 12:30 WIB",Olahraga
4,Bagaimana Olahraga Bisa Menurunkan Gula Darah?...,KOMPAS.com - Olahraga atau aktivitas fisik mer...,"09/10/2024, 12:00 WIB",Olahraga
19,"Siap Jadi Parpol, Bara JP Tunggu Arahan Jokowi","JAKARTA, KOMPAS.com - Barisan Relawan Jokowi P...","10/10/2024, 19:27 WIB",Politik
10,"Terima Kunjungan Elite PKS, Prabowo: Teman Lam...","JAKARTA, KOMPAS.com - Presiden terpilih Prabow...","11/10/2024, 17:13 WIB",Politik
6,"""Event"" Olahraga Bikin Okupansi Hotel di Bali ...","JAKARTA, KOMPAS.com - Diselengarakannya berbag...","08/10/2024, 11:46 WIB",Olahraga
13,"Survei Litbang ""Kompas"": Sektor Politik dan Ke...","JAKARTA, KOMPAS.com - Masyarakat merasa puas d...","11/10/2024, 10:38 WIB",Politik
2,Wanita 54 Tahun Berhasil Turunkan Berat Badan ...,"KOMPAS.com - Dalam hal penurunan berat badan, ...","10/10/2024, 08:45 WIB",Olahraga
